In [1]:
import numpy as np
import qutip as qt
import pandas as pd

from pulsee import simulation as sim, operators as op, spin_squeezing as sqz

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
QUANTUM_NUMBERS = [3/2] # spin 3/2
GAMMA_2PIS = [4.17169] # = gamma / 2pi

In [3]:
def defHlab(eta, omegaQ, Iz, Ix2, Iy2, Iz2, gammaS=0, field=0):
    Hamil = -gammaS*field*Iz
    
    Hamil += omegaQ*((3*Iz2)/2)
    #omegaQ is (uni)axail anisotropy?

    Hamil += omegaQ*eta*(Ix2 - Iy2)/2
    #omegaT transverse anisotropy
    #eta = omegaT/omegaQ, let's keep omegaQ constant
    return [Hamil]

In [4]:
def varyParams(eta, WQ, theta, phi):
    # Place the nulceus in a field
    B0 = 0
    W0 = GAMMA_2PIS[0] * B0 * 2 * np.pi 
    B1 = 1e-10
    zeem_par = {'field magnitude' : B0, 'theta_z' : 0, 'phi_z' : 0}
    
    ACQUISITION_TIME = 800 # in microseconds
    T2 = 500
    
    h_dict = {
    }
    
    args = h_dict.copy()
    spin_par = []
    for qn, gam in zip(QUANTUM_NUMBERS, GAMMA_2PIS):
        spin_par.append({'quantum number': qn, 'gamma/2pi': gam})
    
    quad_par = [{'coupling constant' : 3 * WQ/np.pi,
                        'asymmetry parameter' : 0,
                        'alpha_q' : np.pi,
                        'beta_q' : 0.,
                        'gamma_q' : 0.,
                        'order' : 0}]
    
    args['spin_par'] = spin_par 
    args['zeem_par'] = zeem_par
    args['quad_par'] = quad_par

    pulse_time = 2 / (4 * 3 * GAMMA_2PIS[0] * B1)
    mode = pd.DataFrame([( B0 * GAMMA_2PIS[0], B1, 0., np.pi/2, 0, pulse_time)], 
                           columns=['frequency', 'amplitude', 'phase', 'theta_p', 'phi_p', 'pulse_time'])

    theta = np.deg2rad(theta)
    phi = np.deg2rad(phi)
    
    spin, _, dm_initial = sim.nuclear_system_setup(spin_par, quad_par=quad_par,
                                            zeem_par=zeem_par, initial_state=[{'theta' : theta, 'phi' :phi}])
    sqz_ops = sqz.useful_sqz_ops(spin)
    Ix, Iy, Iz, Ix2, Iy2, Iz2, Ip2, Ip_2Iz = (sqz_ops.Ix, sqz_ops.Iy, sqz_ops.Iz, sqz_ops.Ix2, sqz_ops.Iy2, sqz_ops.Iz2, sqz_ops.Ip2, sqz_ops.Ip_2Iz)

    h_unperturbed = defHlab(0, WQ, Iz, Ix2, Iy2, Iz2)
    
    fig = psplt.plot_complex_density_matrix(dm_initial)
    
    wQaxis = (1/3)/WQ
    times = np.arange(0, wQaxis, 1e-4)

    opts = qt.Options(atol=1e-16, rtol=1e-16, rhs_reuse=False, nsteps=40000)
    dms = sim.evolve(spin, h_unperturbed, dm_initial, solver='mesolve', \
                        mode=mode, times=times, opts=opts, return_allstates=True)

    sqz_ops = sqz.populate_averge_values(dms, sqz_ops)

    It = np.sqrt(sqz_ops.avIx**2+sqz_ops.avIy**2+sqz_ops.avIy**2)
    sqz.plot_values([sqz_ops.avIx, sqz_ops.avIy, sqz_ops.avIz, It], times, [2,2],
                    wQaxis, x_label=r'Scaled Time ($\omega_Q^{-1}$)')


    DIx = np.sqrt(sqz_ops.avIx2-sqz_ops.avIx**2)
    DIy = np.sqrt(sqz_ops.avIy2-sqz_ops.avIy**2)
    DIz = np.sqrt(sqz_ops.avIz2-sqz_ops.avIz**2)
    
    sqz.plot_values([DIx, DIy, DIz], times, 1, wQaxis, title='Standard deviation values', 
                y_label='Standard Deviation', x_label='Scaled Time ($\omega_Q^{-1}$)', labels=['\Delta I_x', '\Delta I_y', '\Delta I_z'])

    xi, alpha, Jn_1, Jn_2, Jn_3 = sqz.calc_squeez_param(sqz_ops, spin.I['I'], xi_sq=True, return_av_spher=True)
    sqz.plot_values([xi, alpha], times, [2, 1], wQaxis, title=r'Gerenalized $\xi^2$',
                    labels=["\\xi^2", "\\alpha_\\xi"], x_label='Scaled Time ($\omega_Q^{-1}$)', put_brackets=False)

    sqz.plot_values([Jn_1, Jn_2, Jn_3], times, 1, wQaxis, title='Mean values of magnetization in spherical',
                     x_label='Scaled Time ($\omega_Q^{-1}$)', labels=['Jn_1', 'Jn_2', 'Jn_3'],put_brackets=True)

#specify the step for 𝜂
step = 0.01

#specify the step for𝜔𝑄
step1 = 1*2*np.pi*1e1

interactive_plot = interactive(varyParams, eta=(0, 1, step), WQ = (0, 30.286*2*np.pi, step1), theta=(0, 180, 10), phi=(0, 360, 10))
output = interactive_plot.children[-1]
output.layout.height = '1380px'
interactive_plot   

interactive(children=(FloatSlider(value=0.0, description='eta', max=1.0, step=0.01), FloatSlider(value=62.8318…